In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
train_dir = '1train'
test_dir = '1test'

In [3]:
input_shape = (48, 48, 3)
batch_size = 32
epochs = 80
num_classes = 7

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 5830 images belonging to 7 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1467 images belonging to 7 classes.


In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
checkpoint = ModelCheckpoint('vgg_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [14]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/5


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


183/183 ━━━━━━━━━━━━━━━━━━━━ 83s 433ms/step - accuracy: 0.2188 - loss: 1.8950 - val_accuracy: 0.3197 - val_loss: 1.6995
Epoch 2/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 53s 288ms/step - accuracy: 0.2831 - loss: 1.7498 - val_accuracy: 0.3279 - val_loss: 1.6967
Epoch 3/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 55s 301ms/step - accuracy: 0.2968 - loss: 1.7258 - val_accuracy: 0.3436 - val_loss: 1.6613
Epoch 4/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 64s 346ms/step - accuracy: 0.3067 - loss: 1.7092 - val_accuracy: 0.3401 - val_loss: 1.6680
Epoch 5/5
183/183 ━━━━━━━━━━━━━━━━━━━━ 58s 317ms/step - accuracy: 0.2996 - loss: 1.7090 - val_accuracy: 0.3456 - val_loss: 1.6749


In [15]:
model.save('final_vgg_model.keras')